In [2]:
# # IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# # RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
# import kagglehub
# kagglehub.login()


In [ ]:
# # IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# # THEN FEEL FREE TO DELETE THIS CELL.
# # NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# # ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# # NOTEBOOK.

# arjunravi26_imdb_dataset_path = kagglehub.dataset_download('arjunravi26/imdb-dataset')

# print('Data source import complete.')


100%|██████████| 25.7M/25.7M [00:02<00:00, 11.2MB/s]

Extracting files...


Data source import complete.


# **Sentimental Analysis**

## Sentimental Analysis using HuggingFace

In [ ]:
# from transformers import pipeline

In [ ]:
# task = 'sentiment-analysis'
# model_name='distilbert/distilbert-base-uncased-finetuned-sst-2-english'
# device = 'cpu'

In [ ]:
# model = pipeline(task=task,model=model_name,device=device)

In [ ]:
# text_positive = ['I Love you']
# text_negative = ['I Hate you']

In [ ]:
# model.predict(text_positive)

In [ ]:
# model.predict(text_negative)

Steps:
1. Load data
    * loda imdb data using pandas
2. Cleaning:
    * Tokenization
    * Remove Stopwords
    * Lemmatization
3. Vectorization
    * Bow
    * TFIDF
    * Word2vec
    * Avgword2vec
    * Embedding tensorflow
    * BERT
4. Train Model
    * SVC
    * Logistic Regressor
    * RNN
    * LSTM
5. Test Model
    * Accuracy
    * precision, Recall
    * confussion metrics

Load Libraries

In [ ]:
import pandas as pd
import gensim
import nltk
import spacy
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout,Embedding,LSTM,SimpleRNN
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download('all',quiet=True)

True

**1. Load data**

In [ ]:
import os

In [2]:
file_path = 'IMDB.csv'

In [ ]:
file_path

'/root/.cache/kagglehub/datasets/arjunravi26/imdb-dataset/versions/1/IMDB.csv'

In [ ]:
imdb = pd.read_csv(file_path)

In [ ]:
imdb.head(3)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive


In [ ]:
imdb.shape

(50000, 2)

In [ ]:
imdb['sentiment'].value_counts()

,count
sentiment,
1,25000
0,25000


**2. Cleaning**

In [ ]:
import re
from bs4 import BeautifulSoup

Remove Noise and stopwords from the text

In [ ]:
def remove_noise(x):
    text = BeautifulSoup(x,'html.parser').get_text()
    text = re.sub(r'http\S+|www\S+|https\S+','',text,flags=re.MULTILINE)
    text = re.sub(r'[^a-zA-Z\s]','',text)
    words = nltk.tokenize.word_tokenize(text)
    stopwords = set(nltk.corpus.stopwords.words('english'))
    return [w.lower() for w in words if w not in stopwords]

In [ ]:
imdb['tokens'] = imdb['review'].apply(remove_noise)

In [ ]:
imdb

,review,sentiment,tokens
0,One of the other reviewers has mentioned that ...,positive,"[one, reviewers, mentioned, watching, oz, epis..."
1,A wonderful little production. <br /><br />The...,positive,"[a, wonderful, little, production, the, filmin..."
2,I thought this was a wonderful way to spend ti...,positive,"[i, thought, wonderful, way, spend, time, hot,..."
3,Basically there's a family where a little boy ...,negative,"[basically, theres, family, little, boy, jake,..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[petter, matteis, love, time, money, visually,..."
...,...,...,...
49995,I thought this movie did a down right good job...,positive,"[i, thought, movie, right, good, job, it, wasn..."
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,"[bad, plot, bad, dialogue, bad, acting, idioti..."
49997,I am a Catholic taught in parochial elementary...,negative,"[i, catholic, taught, parochial, elementary, s..."
49998,I'm going to have to disagree with the previou...,negative,"[im, going, disagree, previous, comment, side,..."


Lematization (Convert word into its root format)

In [ ]:
# nltk.data.path.append('/usr/share/nltk_data')

In [ ]:
nltk.download('omw-1.4', force=True)


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
def lematize_words(x):
    lematizer = nltk.stem.WordNetLemmatizer()
    return [lematizer.lemmatize(w) for w in x]

In [ ]:
imdb['lematized_tokens'] = imdb['tokens'].apply(lematize_words)

In [ ]:
imdb.head(3)

,review,sentiment,tokens,lematized_tokens
0,One of the other reviewers has mentioned that ...,positive,"[one, reviewers, mentioned, watching, oz, epis...","[one, reviewer, mentioned, watching, oz, episo..."
1,A wonderful little production. <br /><br />The...,positive,"[a, wonderful, little, production, the, filmin...","[a, wonderful, little, production, the, filmin..."
2,I thought this was a wonderful way to spend ti...,positive,"[i, thought, wonderful, way, spend, time, hot,...","[i, thought, wonderful, way, spend, time, hot,..."


**3. Vectorization**

BOW

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
countvectorizer = CountVectorizer()
x = countvectorizer.fit_transform(imdb['lematized_tokens'].astype(str))

In [ ]:
# x.toarray()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer()
tfidf.fit_transform(imdb['lematized_tokens'].astype(str))

<50000x203328 sparse matrix of type '<class 'numpy.float64'>'
	with 5058287 stored elements in Compressed Sparse Row format>

In [ ]:
word2vec_model = gensim.models.Word2Vec(vector_size=100,window=3,min_count=5,sentences=imdb['lematized_tokens'])

In [ ]:
word2vec_model.wv['name'] # to see the embedding of word

array([-0.25932387, -0.10123107, -0.858102  , -0.05596836,  1.2835974 ,
       -0.5878436 ,  0.27814886,  1.5754994 ,  1.0669416 ,  0.699311  ,
       -0.27463692,  0.6612554 , -0.9785771 , -1.1170247 ,  0.29426682,
       -0.6936192 , -0.7077677 ,  0.07297418,  0.09088185, -0.34315196,
        0.9038301 ,  0.06468754, -0.34612638, -2.5373797 ,  0.10452145,
        0.12258441,  1.3114038 ,  0.3029611 , -1.196494  , -0.7171911 ,
        2.6839619 ,  0.7569778 ,  0.15103273, -0.12723663,  2.1033442 ,
       -0.9435949 ,  0.4535198 ,  1.3697554 , -0.7327719 , -0.0863288 ,
       -0.95170504, -0.23558247,  1.1930721 , -0.9903567 , -0.9413765 ,
        0.03147724, -0.9982673 ,  1.0749862 , -0.7738949 ,  0.07560628,
        0.2975505 ,  0.03726713,  0.5518456 , -0.8781369 ,  0.44650578,
       -0.01410942, -0.29142597,  0.3488618 , -0.85466576,  1.2505558 ,
       -0.26682508, -1.0912527 ,  0.62309223, -0.26614743, -0.48369166,
       -1.391416  ,  1.2111096 , -1.2242186 , -0.45777956,  0.73

In [ ]:
word2vec_model.wv.similarity('boy','men') # to see the similarity of men and boy

0.38396588

In [ ]:
word2vec_model.wv.most_similar('boy',topn=5) # To get the most similar words

[('girl', 0.8055963516235352),
 ('kid', 0.6704156398773193),
 ('orphan', 0.6221808195114136),
 ('sister', 0.5901929140090942),
 ('nerd', 0.5864043235778809)]

In [ ]:
word2vec_model.wv.doesnt_match(['girl','boy','men','apple']) # Find the odd one

'apple'

In [ ]:
word2vec_model.wv.most_similar(positive=["woman", "queen"], negative=["man"], topn=3) # analogy difference(women is to queen and men is to)

[('princess', 0.6090841293334961),
 ('blonde', 0.5509583950042725),
 ('goddess', 0.5464005470275879)]

Convert word2vec embedding into df columns

In [ ]:
import numpy as np

In [ ]:
def word2vec_embedding(X):
  words = [valid_word for valid_word in X if valid_word in word2vec_model.wv]
  if words:
    return word2vec_model.wv[words].mean(axis=0)
  return np.zeros(100)

In [ ]:
imdb['lematized_embedding'] = imdb['lematized_tokens'].apply(word2vec_embedding)

In [ ]:
imdb['sentiment'] = imdb['sentiment'].map({'positive':1,'negative':0})

In [ ]:
imdb.head()

,review,sentiment,tokens,lematized_tokens,lematized_embedding
0,One of the other reviewers has mentioned that ...,1,"[one, reviewers, mentioned, watching, oz, epis...","[one, reviewer, mentioned, watching, oz, episo...","[-0.21326067, 0.10412378, -0.2077568, 0.010172..."
1,A wonderful little production. <br /><br />The...,1,"[a, wonderful, little, production, the, filmin...","[a, wonderful, little, production, the, filmin...","[-0.08544589, -0.08377791, -0.14477842, -0.175..."
2,I thought this was a wonderful way to spend ti...,1,"[i, thought, wonderful, way, spend, time, hot,...","[i, thought, wonderful, way, spend, time, hot,...","[-0.27350312, -0.061907534, -0.3545318, 0.1584..."
3,Basically there's a family where a little boy ...,0,"[basically, theres, family, little, boy, jake,...","[basically, there, family, little, boy, jake, ...","[-0.43506455, -0.1364409, -0.2991738, 0.016759..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,"[petter, matteis, love, time, money, visually,...","[petter, matteis, love, time, money, visually,...","[-0.16728249, -0.23888566, -0.26214764, 0.1449..."


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
X,y = imdb['lematized_embedding'],imdb['sentiment']

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
X_train

,lematized_embedding
39087,"[-0.25930744, -0.008995485, -0.22912522, 0.023..."
30893,"[-0.22881414, 0.038535893, -0.37045905, 0.2697..."
45278,"[-0.2802092, 0.02837948, -0.61357176, -0.23801..."
16398,"[-0.13347875, 0.13357161, -0.122634485, -0.122..."
13653,"[-0.36557046, -0.28402576, -0.26756543, 0.1677..."
...,...
11284,"[0.022320183, -0.07451757, -0.18591714, -0.023..."
44732,"[-0.5267717, -0.01907233, -0.3759227, 0.182650..."
38158,"[-0.5178055, -0.022388633, -0.2850657, 0.13511..."
860,"[-0.39878976, -0.29691732, -0.52911574, -0.047..."


In [ ]:
X_train = np.vstack(X_train)
X_test = np.vstack(X_test)

In [ ]:
X_train

array([[-0.25930744, -0.00899548, -0.22912522, ...,  0.23625694,
         0.13777189, -0.18841133],
       [-0.22881414,  0.03853589, -0.37045905, ...,  0.4318238 ,
         0.11387586, -0.22310182],
       [-0.2802092 ,  0.02837948, -0.61357176, ...,  0.32920963,
        -0.21573204, -0.33382955],
       ...,
       [-0.5178055 , -0.02238863, -0.2850657 , ...,  0.1586265 ,
        -0.21222724, -0.40262228],
       [-0.39878976, -0.29691732, -0.52911574, ...,  0.11747725,
         0.13496439, -0.33802694],
       [-0.35436216, -0.24621889, -0.18776834, ...,  0.5779548 ,
         0.20636727, -0.3648055 ]], dtype=float32)

Machine Learning

In [ ]:
svc = SVC(kernel='rbf')
svc.fit(X_train,y_train)
y_pred = svc.predict(X_test)
accuracy_score(y_pred,y_test)

0.8586

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
accuracy_score(y_pred,y_test)

0.8517

Deep Learning

RNN

In [ ]:
rnn = Sequential([
    SimpleRNN(units=128,activation='tanh',input_shape=(100,1)),
    Dense(units=1,activation='sigmoid')
    ]
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
rnn.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
rnn.fit(X_train,y_train,epochs=5,batch_size=32,validation_split=0.2)

Epoch 1/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.7784 - loss: 0.4660 - val_accuracy: 0.8239 - val_loss: 0.3974
Epoch 2/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.8076 - loss: 0.4220 - val_accuracy: 0.8284 - val_loss: 0.3860
Epoch 3/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.8217 - loss: 0.4024 - val_accuracy: 0.8339 - val_loss: 0.3825
Epoch 4/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.8291 - loss: 0.3868 - val_accuracy: 0.8418 - val_loss: 0.3723
Epoch 5/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.8302 - loss: 0.3831 - val_accuracy: 0.8314 - val_loss: 0.3753


In [ ]:
rnn.evaluate(X_test,y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8374 - loss: 0.3690


[0.3746708035469055, 0.8319000005722046]

In [ ]:
rnn.predict(X_test[0].reshape(1,100))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step


array([[0.13921484]], dtype=float32)

In [ ]:
pred = rnn.predict(X_test[0].reshape(1,100))
if pred <= 0.5:
  print(f'Negative | Confidence {100 - pred}')
else:
  print(f'Positive | Confidence {pred}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Negative | Confidence [[99.86079]]


LSTM

In [ ]:
lstm = Sequential([
    LSTM(units=128,activation='tanh',input_shape=(100,1)),
    Dense(units=1,activation='sigmoid')
])

In [ ]:
lstm.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
lstm.fit(X_train,y_train,epochs=5,batch_size=5,validation_split=0.2)

Epoch 1/5
6400/6400 ━━━━━━━━━━━━━━━━━━━━ 50s 7ms/step - accuracy: 0.6314 - loss: 0.6413 - val_accuracy: 0.7222 - val_loss: 0.5545
Epoch 2/5
6400/6400 ━━━━━━━━━━━━━━━━━━━━ 81s 7ms/step - accuracy: 0.7476 - loss: 0.5179 - val_accuracy: 0.7847 - val_loss: 0.4611
Epoch 3/5
6400/6400 ━━━━━━━━━━━━━━━━━━━━ 86s 8ms/step - accuracy: 0.7922 - loss: 0.4472 - val_accuracy: 0.8191 - val_loss: 0.4210
Epoch 4/5
6400/6400 ━━━━━━━━━━━━━━━━━━━━ 76s 7ms/step - accuracy: 0.8186 - loss: 0.4041 - val_accuracy: 0.8315 - val_loss: 0.3844
Epoch 5/5
6400/6400 ━━━━━━━━━━━━━━━━━━━━ 44s 7ms/step - accuracy: 0.8264 - loss: 0.3891 - val_accuracy: 0.8379 - val_loss: 0.3673


In [ ]:
lstm.evaluate(X_test,y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8382 - loss: 0.3697


[0.3759971261024475, 0.8379999995231628]

Prediction

In [ ]:
pred = lstm.predict(X_test[0].reshape(1,100))
if pred <= 0.5:
  print(f'Negative | Confidence {100 - pred}')
else:
  print(f'Positive | Confidence {pred}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
Negative | Confidence [[99.847664]]


In [ ]:
def predict(index):
  lstm_pred = lstm.predict(X_test[index].reshape(1,100))
  rnn_pred = rnn.predict(X_test[index].reshape(1,100))
  svc_pred = svc.predict(X_test[index].reshape(1,100))
  lr_pred = lr.predict(X_test[index].reshape(1,100))
  if lstm_pred <= 0.5:
    print('Negative')
    print(f'Negative | Confidence LSTM: {100 - pred} | Confidence RNN: {100 - pred} | SVC {svc_pred} | Logistic Regression {lr_pred}')
  else:
    print(f'Positive | Confidence LSTM: {100 - pred} | Confidence RNN: {100 - pred} | SVC {svc_pred} | Logistic Regression {lr_pred}')

In [ ]:
for i in range(10):
  print(f"Prediction for sample {i} in X_test:")
  predict(i)

Prediction for sample 0 in X_test:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Negative
Negative | Confidence LSTM: [[99.847664]] | Confidence RNN: [[99.847664]] | SVC [0] | Logistic Regression [0]
Prediction for sample 1 in X_test:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Positive | Confidence LSTM: [[99.847664]] | Confidence RNN: [[99.847664]] | SVC [1] | Logistic Regression [1]
Prediction for sample 2 in X_test:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Negative
Negative | Confidence LSTM: [[99.847664]] | Confidence RNN: [[99.847664]] | SVC [0] | Logistic Regression [0]
Prediction for sample 3 in X_test:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Positive | Confidence LSTM: [[99.847664]] | Confidence RNN: [[99.847664]] | SVC [1] | Logistic Regression [1]
Prediction for sample 4 in X_test:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 